# A review on Rule extraction with Anchors

Sophisticated machine learning models such as deep neural
networks have been shown to be highly accurate for many
applications, even though their complexity virtually makes
them black-boxes.

basicly anchors are "if then" rules in a classification tasks that we can find using reanforcment learning and graph search aglorithms.
the anchors approach deploys a perturbation-based strategy to generate local explanations for predictions of black box machine learning models. However, instead of surrogate models used by LIME, the resulting explanations are expressed as easy-to-understand IF-THEN rules, called anchors. <br> Finding anchors involves an exploration or multi-armed bandit problem, which originates in the discipline of reinforcement learning. To this end, neighbors, or perturbations, are created and evaluated for every instance that is being explained. Doing so allows the approach to disregard the black box’s structure and its internal parameters so that these can remain both unobserved and unaltered. Thus, the algorithm is model-agnostic, meaning it can be applied to any class of model.

<center><img src="https://christophm.github.io/interpretable-ml-book/images/anchors-visualization.png" width="500"></center>

Anchors are intuitive, easy to comprehend, and
have extremely clear coverage – they only apply when all the
conditions in the rule are met, and if they apply the precision
is high (by design).

### Explanation:

given a black box model $f : X \rightarrow Y$ and an instance $x \in X$.

$A$ be a rule (set of predicates) acting on such an in-
terpretable representation, such that $A(x)$ returns $1$ if all its
feature predicates are true for instance $x$. 

A is an anchor if:

$$ \exists_D(z|A) [1_f(x)=f(z)] >= t, A(x) = 1$$

*$x$
represents the instance being explained (e.g. one row in a tabular data set).

*$A$
is a set of predicates, i.e., the resulting rule or anchor, such that A(x)=1 when all feature predicates defined by $A$ correspond to $x$
’s feature values.

*$f$
denotes the classification model to be explained (e.g. an artificial neural network model). It can be queried to predict a label for $x$
and its perturbations.

*$D_x(⋅|A)$
indicates the distribution of neighbors of $x$, matching $A$
.

*$0≤τ≤1$
specifies a precision threshold. Only rules that achieve a local fidelity of at least $τ$ are considered a valid result.



### Finding Anchors

Although anchors’ mathematical description may seem clear and straightforward, constructing particular rules is infeasible. It would require evaluating $1_{\hat{f(x)}=\hat{f(z)}}$ for all $z∈Dx(⋅|A)$ which is not possible in continuous or large input spaces. Therefore, the authors propose to introduce the parameter $0≤δ≤1$

to create a probabilistic definition. This way, samples are drawn until there is statistical confidence concerning their precision. The probabilistic definition reads as follows:

$P(prec(A)≥τ)≥1−δwithprec(A)=EDx(z|A)[1_{\hat{f(x)}=\hat{f(z)}}]$

The previous two definitions are combined and extended by the notion of coverage. Its rationale consists of finding rules that apply to a preferably large part of the model’s input space. Coverage is formally defined as an anchor’s probability of applying to its neighbors, i.e. its perturbation space:

$cov(A)=ED(z)[A(z)]$

Including this element leads to anchor’s final definition taking into account the maximization of coverage:

$maxAs.t.P(prec(A)≥τ)≥1−δcov(A)$

Thus, the proceeding strives for a rule that has the highest coverage among all eligible rules (all those that satisfy the precision threshold given the probabilistic definition). These rules are thought to be more important, as they describe a larger part of the model. Note that rules with more predicates tend to have higher precision than rules with fewer predicates. In particular, a rule that fixes every feature of x
reduces the evaluated neighborhood to identical instances. Thus, the model classifies all neighbors equally, and the rule’s precision is 1

. At the same time, a rule that fixes many features is overly specific and only applicable to a few instances. Hence, there is a trade-off between precision and coverage.

The anchors approach uses four main components to find explanations.

Candidate Generation: Generates new explanation candidates. In the first round, one candidate per feature of $x$

gets created and fixes the respective value of possible perturbations. In every other round, the best candidates of the previous round are extended by one feature predicate that is not yet contained therein.

Best Candidate Identification: Candidate rules are to be compared in regard to which rule explains $x$

the best. To this end, perturbations that match the currently observed rule are created and evaluated by calling the model. However, these calls need to be minimized as to limit computational overhead. This is why, at the core of this component, there is a pure-exploration Multi-Armed-Bandit (MAB; KL-LUCB62, to be precise). MABs are used to efficiently explore and exploit different strategies (called arms in an analogy to slot machines) using sequential selection. In the given setting, each candidate rule is to be seen as an arm that can be pulled. Each time it is pulled, respective neighbors get evaluated, and we thereby obtain more information about the candidate rule’s payoff (precision in anchor’s case). The precision thus states how well the rule describes the instance to be explained.

Candidate Precision Validation: Takes more samples in case there is no statistical confidence yet that the candidate exceeds the $τ$

threshold.

Modified Beam Search: All of the above components are assembled in a beam search, which is a graph search algorithm and a variant of the breadth-first algorithm. It carries the $B$
best candidates of each round over to the next one (where B is called the Beam Width). These B best rules are then used to create new rules. The beam search conducts at most $featureCount(x)$ rounds, as each feature can only be included in a rule at most once. Thus, at every round $i$, it generates candidates with exactly i predicates and selects the B best thereof. Therefore, by setting $B$

high, the algorithm is more likely to avoid local optima. In turn, this requires a high number of model calls and thereby increases the computational load.

These four components are shown in the figure below.

<center> <img src="https://christophm.github.io/interpretable-ml-book/images/anchors-process.jpg" ></center>

Beam search sudo code:

 <img src="Beam.png" width="450"> 
 


### Time Complexity
 
 Let $B$ denote the beam width and $p$ the number of all features. Then the anchors algorithm is subject to:
 
 $$O(B⋅p^2+p^2⋅OMAB[B⋅p,B])$$

### OpenSource Notebooks:
*use anchors for image classification: <a href="https://github.com/marcotcr/anchor/blob/master/notebooks/Image%20example%20with%20torchvision.ipynb"> Link </a>

*use anchors for text: <a href="https://github.com/marcotcr/anchor/blob/master/notebooks/Anchor%20for%20text.ipynb"> Link </a>

### Python pakage
https://github.com/marcotcr/anchor

### sourses: 

1 - Anchors: High-Precision Model-Agnostic Explanations, <a href="https://homes.cs.washington.edu/~marcotcr/aaai18.pdf"> Link </a> <br>
2 - interpretable learning (A Guide for Making Black Box Models Explainable) - Christoph Molnar <a href = "https://christophm.github.io/interpretable-ml-book/index.html"> Link </a>